In [59]:
%load_ext autoreload
%autoreload 2
import sys
import keras
from keras.models import Model
from keras.layers import Conv2D, UpSampling2D
sys.path.insert(0, "../scripts/")
from load_encoder_decoder import build_encoder_decoder_from_vgg
from data_generator import DataGenerator
from losses import overall_loss_wrapper, sad_wrapper, mse_wrapper, alpha_loss_wrapper
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import multi_gpu_model
from keras.models import load_model
from unpooling import Unpooling
from load_refinement import build_refinement

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
model = load_model("../model_checkpoints/encoder_decoder.09-val_loss-0.1228-val_sad-176.7991-val_mse-0.0449.hdf5", custom_objects={"Unpooling": Unpooling()})

C:\Users\thalv\Anaconda3\envs\tensorflow-cpu\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [24]:
for layer in model.layers:
        layer.trainable = False

In [25]:
refinement = build_refinement(model)

In [7]:
PATIENCE=30
PATH_MODEL_CHECKPOINTS = "../model_checkpoints/"
PATH_LOGS = "../logs/"
BACKGROUNDS_PER_FG_TRAIN = 100
BACKGROUNDS_PER_FG_TEST = 20
BATCH_SIZE=2

In [21]:
fg_train = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Training_set\\all_foregrounds\\"
bg_train = "D:\\Image Matting Dataset\\mscoco\\train2014\\train2014\\"
a_train = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Training_set\\all_alphas\\"

fg_names_train = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Training_set\\training_fg_names.txt"
bg_names_train = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Training_set\\training_bg_names.txt"


fg_test = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\Adobe-licensed images\\fg\\"
bg_test = "D:\\Image Matting Dataset\\mscoco\\test_selected\\"
a_test = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\Adobe-licensed images\\alpha\\"

fg_names_test = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\test_fg_names.txt"
bg_names_test = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\test_bg_names.txt"

# test_gen = DataGenerator(path_combined_test, path_alphas_test, 16, False) 
train_gen = DataGenerator(fg_names_train, bg_names_train, BACKGROUNDS_PER_FG_TRAIN, fg_train, bg_train, a_train, BATCH_SIZE, False)
test_gen = DataGenerator(fg_names_test, bg_names_test, BACKGROUNDS_PER_FG_TEST, fg_test, bg_test, a_test, BATCH_SIZE, False)

In [9]:
tensor_board = keras.callbacks.TensorBoard(log_dir=PATH_LOGS, histogram_freq=0, write_graph=True, write_images=True)
model_name = PATH_MODEL_CHECKPOINTS + 'encoder_decoder.{epoch:02d}-val_loss-{val_loss:.4f}-val_sad-{val_sad:.4f}-val_mse-{val_mse:.4f}.hdf5'
model_checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True)
early_stop = EarlyStopping('val_loss', patience=PATIENCE, verbose=1)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=PATIENCE // 4, verbose=1)

callbacks = [tensor_board, model_checkpoint, early_stop, reduce_lr]

In [214]:
inputs = refinement.input
refinement.compile(optimizer="adam", loss=alpha_loss_wrapper(inputs), metrics=[sad_wrapper(inputs), mse_wrapper(inputs)])

In [215]:
history = refinement.fit_generator(train_gen, validation_data=test_gen, use_multiprocessing=False, workers=1, callbacks=callbacks, shuffle=False, epochs=1, validation_steps=2, steps_per_epoch=2)

Epoch 1/1
2/2 [==============================] - 39s 20s/step - loss: 1.6430 - sad: 30.2286 - mse: 0.0502 - val_loss: 1.9888 - val_sad: 82.9653 - val_mse: 0.3268

Epoch 00001: val_loss did not improve from 1.43819
